<a href="https://colab.research.google.com/github/ahcamachod/1893-word2vec-interpretacion-del-lenguaje-humano-con-word-embedding/blob/aula-4/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: Interpretación del lenguaje humano con Word Embedding

En este notebook desarrollaremos un clasificador de noticias para una start-up de noticias llamada **Alura Latam News**.

El modelo Word2Vec utilizado para entrenar nuestro clasificador fue tomado de la siguiente fuente: 
 
Aitor Almeida, Aritz Bilbao Jayo. (2018) "Word2vec models for the Spanish Language." Available from: https://github.com/aitoralmeida/spanish_word2vec

## 1. Explorando el Dataset 

In [1]:
import pandas as pd

entrenamiento = pd.read_csv("/content/drive/MyDrive/word2vec/noticias_entrenamiento.csv")
test = pd.read_csv("/content/drive/MyDrive/word2vec/noticias_prueba.csv")
entrenamiento.sample(5)

,fecha,titulo,pais,extracto,resumen,enlace,categoria
86741,2022-03-18 11:15:00,Occidente pone en riesgo su seguridad al armar...,PR,"El embajador ruso ante la ONU, Vasili Nebenzia...","El embajador ruso ante la ONU, Vasili Nebenzia...",https://www.noticel.com/mundo/top-stories/2022...,mundo
65498,2022-03-20 17:41:00,Rodríguez destacó el potencial de la producció...,AR,"El ministro de Desarrollo Agrario bonaerense, ...","El ministro de Desarrollo Agrario bonaerense, ...",https://www.eldestapeweb.com/economia/buenos-a...,economia
55145,2022-03-31 14:58:11,"""El Gobierno puede garantizar que no va a falt...",AR,"La portavoz presidencial indicó que este 2022,...","La portavoz de la Presidencia, Gabriela Cerrut...",https://www.baenegocios.com/negocios/El-Gobier...,economia
27460,2022-04-04 22:30:00,"El rosado, la tendencia 2022 que Eiza González...",AR,"Si te tienes que comprar una prenda, entonces ...",Eiza González está siempre al salto de las ten...,https://www.mdzol.com/mdz-femme/moda/2022/4/4/...,ciencia y tecnologia
13367,2022-03-29 11:02:40,Lorenzo Milá presenta la nueva etapa de 'En po...,ES,El veterano espacio se emitirá los miércoles e...,'En portada' inicia una nueva etapa con varios...,https://www.elperiodico.com/es/yotele/20220329...,entretenimiento


In [2]:
test.sample(5)

,fecha,titulo,pais,extracto,resumen,enlace,categoria
21495,2022-03-18 23:24:12,¿Cómo será la inauguración del AIFA? Sedena ex...,GT,"El titular de la Sedena, Luis Cresencio Sandov...",Este 21 de marzo será la inauguración del Aero...,https://www.tvazteca.com/aztecanoticias/notas/...,politica
8806,2022-03-31 21:59:17,Aida Victoria critica a Marbelle por comentari...,CO,El trino 'racista' de Marbelle a Francia Márqu...,"El martes 29 de marzo, la cantante Maureen Bel...",https://www.rcnradio.com/entretenimiento/aida-...,entretenimiento
4784,2022-04-05 00:35:04,Video: Olivia Rodrigo tira su Grammy y lo rompe,US,La cantante Olivia Rodrigo ganó tres gramófono...,La cantante Olivia Rodrigo ganó tres gramófono...,https://www.dallasnews.com/espanol/al-dia/espe...,entretenimiento
667,2022-04-03 23:14:07,Lista de ganadores de los premios Grammy,US,Lista parcial de ganadores de los premios Gram...,Lista parcial de ganadores de los premios Gram...,https://www.milfordmirror.com/entertainment/ar...,entretenimiento
4386,2022-04-03 14:00:29,Ciencia en Panamá: Ciencia a bordo de mini sub...,PA,Los vehículos sumergibles han permitido a los ...,Por Helkin Guevara (Corresponsal digital)\n\n\...,https://www.tvn-2.com/contenido_exclusivo/Cien...,entretenimiento


In [3]:
entrenamiento.shape

(91844, 7)

In [4]:
test.shape

(22961, 7)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()

In [6]:
texto = [
         "tenga un buen día",
         "tenga un excelente día",
         "tenga un pésimo día luna sol carro casa familia"
]

In [7]:
vector.fit(texto)

CountVectorizer()

In [8]:
vector.vocabulary_

{'buen': 0,
 'carro': 1,
 'casa': 2,
 'día': 3,
 'excelente': 4,
 'familia': 5,
 'luna': 6,
 'pésimo': 7,
 'sol': 8,
 'tenga': 9,
 'un': 10}

In [9]:
vector_bueno = vector.transform(['buen'])

In [10]:
vector_luna = vector.transform(['luna'])

In [11]:
print(vector_bueno)

  (0, 0)	1


In [12]:
vector_bueno

<1x11 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [13]:
print(vector_bueno.toarray())

[[1 0 0 0 0 0 0 0 0 0 0]]


In [14]:
print(vector_luna.toarray())

[[0 0 0 0 0 0 1 0 0 0 0]]


## 2. Cargando un modelo Word2Vec

In [15]:
!unzip '/content/drive/MyDrive/word2vec/keyed_vectors.zip'

Archive:  /content/drive/MyDrive/word2vec/keyed_vectors.zip
  inflating: complete.kv             
  inflating: complete.kv.vectors.npy  


In [16]:
from gensim.models import KeyedVectors

modelo = KeyedVectors.load('/content/complete.kv')

## 3. Entendiendo Word2Vec

In [17]:
modelo.get_vector('rusia')

array([ 1.81506336e-01,  7.12285161e-01, -7.40406871e-01,  2.59009194e+00,
        2.42028356e+00,  3.58315766e-01, -2.15411949e+00,  6.19949877e-01,
       -1.58195674e+00,  1.54491818e+00,  1.46719420e+00, -7.03221202e-01,
        2.09037781e+00, -1.13215275e-01, -5.64340353e-01,  1.01796961e+00,
       -1.66986406e+00, -7.41993561e-02, -3.33077788e-01,  2.32870013e-01,
       -8.01489234e-01,  4.48490649e-01, -4.02902395e-01, -2.12651685e-01,
       -9.53191996e-01, -1.09190546e-01, -3.18247795e-01,  1.79606867e+00,
        2.32772017e+00, -1.92491317e+00,  6.01871610e-01, -5.39401770e-01,
        6.07950449e-01,  2.92399406e-01, -6.02105737e-01,  1.31095541e+00,
       -6.24053299e-01, -5.23794055e-01,  1.00642741e+00, -5.95752716e-01,
        5.76731563e-03, -8.13566089e-01, -2.50728965e+00, -8.18064511e-01,
        2.18556118e+00, -3.08079720e+00,  7.81908751e-01,  6.62506402e-01,
       -8.95268857e-01, -1.91720939e+00, -2.33609056e+00,  5.78275084e-01,
       -1.27308249e+00,  

In [18]:
len(modelo.get_vector('rusia'))

400

In [19]:
modelo.most_similar('rusia')

[('alemania', 0.8213340044021606),
 ('polonia', 0.8001416921615601),
 ('ucrania', 0.7762457132339478),
 ('francia', 0.7627276182174683),
 ('urss', 0.7625904083251953),
 ('europa', 0.7537119388580322),
 ('italia', 0.712181568145752),
 ('turquía', 0.7068266272544861),
 ('españa', 0.7036899328231812),
 ('moscú', 0.702022910118103)]

In [20]:
modelo.most_similar('peugeot')

[('audi', 0.8497860431671143),
 ('citroën', 0.8487632870674133),
 ('bmw', 0.8417755961418152),
 ('toyota', 0.8200501799583435),
 ('porsche', 0.8196049928665161),
 ('chevrolet', 0.8028586506843567),
 ('opel', 0.7941076755523682),
 ('volvo', 0.7930978536605835),
 ('renault', 0.7916873693466187),
 ('volkswagen', 0.7833201885223389)]

In [21]:
modelo.most_similar('portugal')

[('lisboa', 0.7357922792434692),
 ('francia', 0.7306839823722839),
 ('españa', 0.6988375186920166),
 ('italia', 0.6697657108306885),
 ('castilla', 0.6617139577865601),
 ('austria', 0.6451452970504761),
 ('brasil', 0.6380101442337036),
 ('galicia', 0.6311914920806885),
 ('flandes', 0.6305932402610779),
 ('marruecos', 0.6281476020812988)]

In [22]:
modelo.most_similar(positive=['colombia','ecuador'])

[('venezuela', 0.702144980430603),
 ('bolivia', 0.6926755905151367),
 ('peru', 0.6321871876716614),
 ('bogotá', 0.5886013507843018),
 ('perú', 0.5810149908065796),
 ('panamá', 0.5775487422943115),
 ('ecuatoriana', 0.577391505241394),
 ('nicaragua', 0.5680162906646729),
 ('guayaquil', 0.5547071695327759),
 ('paraguay', 0.5499962568283081)]

In [23]:
modelo.most_similar(positive=['paraguay','canada'])

[('canadá', 0.610359787940979),
 ('brasil', 0.6053897738456726),
 ('uruguay', 0.5750980377197266),
 ('argentina', 0.5463083386421204),
 ('perú', 0.5372594594955444),
 ('bolivia', 0.529151201248169),
 ('chile', 0.5125446319580078),
 ('ottawa', 0.4764121472835541),
 ('panamá', 0.4652531147003174),
 ('venezuela', 0.4532638192176819)]

In [24]:
modelo.most_similar(positive=['italia','ferrari'])

[('milán', 0.600520133972168),
 ('francia', 0.5976483225822449),
 ('maserati', 0.5933842658996582),
 ('imola', 0.5910633206367493),
 ('maranello', 0.585680365562439),
 ('sicilia', 0.568071186542511),
 ('vallelunga', 0.5674858689308167),
 ('scuderia', 0.5674735307693481),
 ('bugatti', 0.5651099681854248),
 ('nuvolari', 0.5624744296073914)]

In [25]:
modelo.most_similar(negative=['italia','ferrari'])

[('kenbugul', 0.25659114122390747),
 ('llenaba', 0.25351476669311523),
 ('«armadura»', 0.24964860081672668),
 ('protegía', 0.2473001331090927),
 ('liberaba', 0.23972740769386292),
 ('llenaban', 0.23930209875106812),
 ('oprimía', 0.2392035871744156),
 ('arrojaba', 0.23916885256767273),
 ('śāyin', 0.23895759880542755),
 ('bubónicos', 0.23644055426120758)]

nube : nubes :: estrella : estrellas

nubes + estrella - nube = estrellas 

carro : transporte :: casa : vivienda

In [26]:
modelo.most_similar(positive=['nubes','estrella'], negative=['nube'])

[('estrellas', 0.7377859354019165),
 ('constelaciones', 0.5660182237625122),
 ('pléyades', 0.5638790130615234),
 ('lunas', 0.5301465392112732),
 ('galaxias', 0.520189106464386),
 ('estrellas—', 0.49540942907333374),
 ('estrellitas', 0.49305015802383423),
 ('cumbres', 0.4923092722892761),
 ('cimas', 0.4908401072025299),
 ('auroras', 0.48563718795776367)]

In [27]:
modelo.most_similar(positive=['mujer','profesor'], negative=['hombre'])

[('profesora', 0.7569466829299927),
 ('alumna', 0.641080915927887),
 ('catedrática', 0.5854173898696899),
 ('secretaria', 0.5741715431213379),
 ('psicóloga', 0.564406156539917),
 ('institutriz', 0.5636638402938843),
 ('amiga', 0.5633606910705566),
 ('compañera', 0.5549027919769287),
 ('becaria', 0.5541263818740845),
 ('bibliotecaria', 0.5442808866500854)]

In [28]:
modelo.most_similar(positive=['mujer','médico'], negative=['hombre'])

[('enfermera', 0.727431058883667),
 ('comadrona', 0.6748769283294678),
 ('psicóloga', 0.6288493871688843),
 ('criada', 0.6172555685043335),
 ('ginecóloga', 0.6131819486618042),
 ('sirvienta', 0.6024647951126099),
 ('partera', 0.588049590587616),
 ('chica', 0.5842192769050598),
 ('esposa', 0.5792689919471741),
 ('niñera', 0.5781521797180176)]

## 4. Combinando Vectores

## 5. Clasificando textos